In [2]:
import ffn

# pip install pandas-datareader --upgrade

/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


In [3]:
returns = ffn.get('aapl,msft,dkng,nvda,lmnd', start='2010-01-01').to_returns().dropna()
returns.calc_mean_var_weights().as_format('.2%')

aapl    52.01%
msft     0.00%
dkng    20.68%
nvda     6.72%
lmnd    20.59%
dtype: object

In [4]:
from empyrical import alpha_beta
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import discrete_allocation
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import matplotlib as pyplot
import numpy as np
import pandas as pd

In [186]:
import csv
def get_tickers(year=2016):
    reader = csv.reader(open("TICKER_LIST_CLEANED.csv"))
    next(reader)
    stocks = [line[0].strip() for line in reader if all(punc not in line[0] for punc in ['.','/']) and (not line[7] or int(line[7]) < year)]
    return stocks

In [216]:
tickers = get_tickers(year=2020)
print(tickers[41])
# del tickers[41]
print(tickers[41])
print(tickers[0:52])

ACET
ACET
['A', 'AA', 'AACG', 'AAIC', 'AAL', 'AAMC', 'AAME', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAT', 'AAU', 'AAWW', 'AAXN', 'AB', 'ABB', 'ABBV', 'ABC', 'ABCB', 'ABEO', 'ABEV', 'ABG', 'ABIO', 'ABM', 'ABMD', 'ABR', 'ABT', 'ABTX', 'ABUS', 'AC', 'ACA', 'ACAD', 'ACAM', 'ACAMU', 'ACB', 'ACBI', 'ACC', 'ACCO', 'ACEL', 'ACER', 'ACET', 'ACGL', 'ACGLO', 'ACGLP', 'ACH', 'ACHC', 'ACHV', 'ACIA', 'ACIU', 'ACIW', 'ACKI']


In [215]:
prices = ffn.get(tickers[0:51], start = '2016-01-01')

In [190]:
# prices = ffn.get('msft,aapl,amzn,lmnd,nvda,crsp,tsla', start='2016-01-01')
benchmark = ffn.get('spy', start='2016-01-01')


In [191]:
msft = ffn.get('msft', start='2016-01-01')
ax = prices.rebase().plot()

In [192]:
returns = prices.to_returns().dropna()
stats = prices.calc_stats()

stats.display()

/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packag

Stat                 a           aa          aacg        aaic        aal         aamc        aame        aaoi        aaon        aap         aapl        aat         aau         aaww        aaxn        ab          abb         abbv        abc         abcb        abeo        abev        abg         abio        abm         abmd        abr         abt         abtx        abus        ac          aca         acad        acam        acamu       acb
-------------------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------
Start                2019-05-09  2019-05-09  2019-05-09  2019-05-09  2019-05-09  2019-05-09  2019-05-09

In [193]:
alpha, beta = alpha_beta(msft, benchmark)
print(beta)

1.0318531639121846


/home/dheeraj/.local/lib/python3.6/site-packages/empyrical/stats.py:1384: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  returns = returns[:, np.newaxis]
/home/dheeraj/.local/lib/python3.6/site-packages/empyrical/stats.py:1387: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  factor_returns = factor_returns[:, np.newaxis]
/home/dheeraj/.local/lib/python3.6/site-packages/empyrical/stats.py:1294: RuntimeWarning: overflow encountered in power
  out=out


In [194]:
returns = prices.pct_change()
 
# mean daily return and covariance of daily returns
# mean_daily_returns = returns.mean()
# cov_matrix = returns.cov()
 
# # portfolio weights
# weights = np.asarray([0.4,0.2,0.1,0.1,0.1,0.1])
 
# portfolio_return = round(np.sum(mean_daily_returns * weights) * 252,2)
# portfolio_std_dev = round(np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(252),2)

# print("Expected annualised return: " + str(portfolio_return))
# print("Volatility: " + str(portfolio_std_dev))

In [211]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)


OrderedDict([('a', 0.0), ('aa', 0.0), ('aacg', 0.0), ('aaic', 0.0), ('aal', 0.0), ('aamc', 0.0), ('aame', 0.0), ('aaoi', 0.0), ('aaon', 0.0), ('aap', 0.0), ('aapl', 0.50052), ('aat', 0.0), ('aau', 0.0), ('aaww', 0.0), ('aaxn', 0.0), ('ab', 0.0), ('abb', 0.0), ('abbv', 0.0), ('abc', 0.0), ('abcb', 0.0), ('abeo', 0.0), ('abev', 0.0), ('abg', 0.01042), ('abio', 0.0), ('abm', 0.0), ('abmd', 0.0), ('abr', 0.0), ('abt', 0.0), ('abtx', 0.0), ('abus', 0.0), ('ac', 0.0), ('aca', 0.0), ('acad', 0.04178), ('acam', 0.40552), ('acamu', 0.02086), ('acb', 0.0), ('acbi', 0.0), ('acc', 0.0), ('acco', 0.0), ('acel', 0.0), ('acer', 0.0), ('acet', 0.0), ('acgl', 0.0), ('acglo', 0.0), ('acglp', 0.0), ('ach', 0.0), ('achc', 0.0), ('achv', 0.0), ('acia', 0.02089), ('aciu', 0.0), ('aciw', 0.0)])
Expected annual return: 44.7%
Annual volatility: 21.8%
Sharpe Ratio: 1.96


(0.44734805821921664, 0.21791586573463523, 1.961069042763574)

In [212]:
latest_prices = get_latest_prices(prices)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=6000)
allocation, leftover = da.lp_portfolio()

print(allocation)
print("Funds remaining: ${:.2f}".format(leftover))

{'aapl': 24, 'acad': 5, 'acam': 222, 'acamu': 10, 'acia': 2}
Funds remaining: $2.54


In [209]:
ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
ef.efficient_return(target_return=0.2, market_neutral=True)

OrderedDict([('a', 0.0315015893389542),
             ('aa', -0.0092510905889794),
             ('aacg', -0.0063817376487219),
             ('aaic', -0.0055235677012912),
             ('aal', -0.0162371185431683),
             ('aamc', 0.0012143421988103),
             ('aame', -0.0025862487640527),
             ('aaoi', -0.0083601645867665),
             ('aaon', -0.0170365745519976),
             ('aap', -0.0330210264430172),
             ('aapl', 0.1068110333404698),
             ('aat', -0.0452762597296056),
             ('aau', 0.0018391952747618),
             ('aaww', -0.0087653913958821),
             ('aaxn', 0.0090508193847637),
             ('ab', -0.0182116745145118),
             ('abb', 0.0350227676920341),
             ('abbv', 0.0072254169914589),
             ('abc', 0.0116067353851758),
             ('abcb', 0.0407764456776654),
             ('abeo', -0.0044405015864341),
             ('abev', -0.0265045069387282),
             ('abg', 0.0420824059241794),
            

# Script For Portfolio Optimization Based on Efficient Frontier

In [217]:
prices = ffn.get(tickers[0:51], start = '2016-01-01')
benchmark = ffn.get('spy', start='2016-01-01')

In [218]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)


OrderedDict([('a', 0.0), ('aa', 0.0), ('aacg', 0.0), ('aaic', 0.0), ('aal', 0.0), ('aamc', 0.0), ('aame', 0.0), ('aaoi', 0.0), ('aaon', 0.0), ('aap', 0.0), ('aapl', 0.50052), ('aat', 0.0), ('aau', 0.0), ('aaww', 0.0), ('aaxn', 0.0), ('ab', 0.0), ('abb', 0.0), ('abbv', 0.0), ('abc', 0.0), ('abcb', 0.0), ('abeo', 0.0), ('abev', 0.0), ('abg', 0.01042), ('abio', 0.0), ('abm', 0.0), ('abmd', 0.0), ('abr', 0.0), ('abt', 0.0), ('abtx', 0.0), ('abus', 0.0), ('ac', 0.0), ('aca', 0.0), ('acad', 0.04178), ('acam', 0.40552), ('acamu', 0.02086), ('acb', 0.0), ('acbi', 0.0), ('acc', 0.0), ('acco', 0.0), ('acel', 0.0), ('acer', 0.0), ('acet', 0.0), ('acgl', 0.0), ('acglo', 0.0), ('acglp', 0.0), ('ach', 0.0), ('achc', 0.0), ('achv', 0.0), ('acia', 0.02089), ('aciu', 0.0), ('aciw', 0.0)])
Expected annual return: 44.7%
Annual volatility: 21.8%
Sharpe Ratio: 1.96


(0.44734805821921664, 0.21791586573463523, 1.961069042763574)

In [219]:
latest_prices = get_latest_prices(prices)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=6000)
allocation, leftover = da.lp_portfolio()

print(allocation)
print("Funds remaining: ${:.2f}".format(leftover))

{'aapl': 24, 'acad': 5, 'acam': 222, 'acamu': 10, 'acia': 2}
Funds remaining: $2.54


In [220]:
ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
ef.efficient_return(target_return=0.2, market_neutral=True)

OrderedDict([('a', 0.0320959711433153),
             ('aa', -0.0082949581422592),
             ('aacg', -0.0056937799852446),
             ('aaic', -0.0051046399066623),
             ('aal', -0.0168549249431155),
             ('aamc', 0.0010569000778719),
             ('aame', -0.0026511333462597),
             ('aaoi', -0.0076837344408727),
             ('aaon', -0.0168429900721064),
             ('aap', -0.0328856489498937),
             ('aapl', 0.1075952985090302),
             ('aat', -0.0458436836466612),
             ('aau', 0.0018030763374588),
             ('aaww', -0.0086462165897094),
             ('aaxn', 0.0087948665317227),
             ('ab', -0.0180306143798567),
             ('abb', 0.0331066071206817),
             ('abbv', 0.0086117796551145),
             ('abc', 0.0110946115044426),
             ('abcb', 0.0410193761847629),
             ('abeo', -0.0040418597867516),
             ('abev', -0.024845834481656),
             ('abg', 0.042082093313571),
             (